In [31]:
import sys
print("python version {}".format(sys.version))

python version 3.10.14 (main, May  6 2024, 14:42:37) [Clang 14.0.6 ]


In [1]:
!pip install python-dotenv

# CG Notes / Updates: 
1. .env approach for creating Session object (optional, env specific)
2. validate installation and import of snowpark modin module
3. demonstrate use of modin.pandas to  generate Snowpark Dataframe, locally

storing creds in an .env file

In [2]:
from dotenv import load_dotenv
import os

In [8]:
from snowflake.snowpark import Session

In [3]:
os.getcwd()

'/Users/cgoyette/Documents/GitHub/feature_store'

In [4]:
#load from local .env file
load_dotenv(".env")


True

In [5]:

os.getenv('SNOWFLAKE_USER')

'cgoyette'

In [6]:

connection_details = {
  "account":  os.environ["SNOWFLAKE_ACCOUNT"],
  "user": os.environ["SNOWFLAKE_USER"],
  "password": os.environ["SNOWFLAKE_USER_PASSWORD"],
  "role": os.environ["SNOWFLAKE_ROLE"],
}

In [9]:

session = Session.builder.configs(connection_details).create()

# 1. Import libraries

In [1]:
# Install the Snowpark Python package into the Python virtual environment (automatically installs the appropriate version of PyArrow)
# pip install snowflake-snowpark-python
# ! pip install "snowflake-snowpark-python[modin]"
# pip install modin

In [1]:
! pip install "snowflake-snowpark-python[modin]"

  Using cached modin-0.28.1-py3-none-any.whl.metadata (17 kB)
  Using cached pandas-2.2.1-cp310-cp310-macosx_11_0_arm64.whl.metadata (19 kB)
  Using cached numpy-1.26.4-cp310-cp310-macosx_11_0_arm64.whl.metadata (61 kB)
  Using cached pyarrow-17.0.0-cp310-cp310-macosx_11_0_arm64.whl.metadata (3.3 kB)
Using cached modin-0.28.1-py3-none-any.whl (1.2 MB)
Using cached pandas-2.2.1-cp310-cp310-macosx_11_0_arm64.whl (11.3 MB)
Using cached numpy-1.26.4-cp310-cp310-macosx_11_0_arm64.whl (14.0 MB)
Using cached pyarrow-17.0.0-cp310-cp310-macosx_11_0_arm64.whl (27.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.2
    Uninstalling numpy-2.1.2:
      Successfully uninstalled numpy-2.1.2
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.3
    Uninstalling pandas-2.2.3:
      Successfully uninstalled pandas-2.2.3


# 2. Create Snowflake snowpark sesssion 
#### with SSO through a web browser

In [2]:
print(session)

<snowflake.snowpark.session.Session: account="ska58151", role="FEATLK_NONPROD_DEVELOPER_ROLE", database="FEATLK_DEV", schema="FEATLK_DATA", warehouse="FEATLK_INGEST_DEV">


In [ ]:
# # If you have configured Snowflake to use single sign-on (SSO), you can configure your client application to use browser-based SSO for authentication.
import json
from snowflake.snowpark import Session
connection_parameters = json.load(open('connection.json'))
print(connection_parameters)
# connect to the database
session = Session.builder.configs(connection_parameters).create()


In [10]:
import modin.pandas as pd

In [11]:
import snowflake.snowpark.modin.plugin

## NOTE: Couldn't resolve
ModuleNotFoundError: No module named 'snowflake.snowpark.modin'

Can't i run `snowflake pandas` out side of snowflake notebooks? 

### CG note

Yes, you can use modin.pandas locally
Ref: (https://docs.snowflake.com/developer-guide/snowpark/python/pandas-on-snowflake)

However, in the cell below, you've re-imported pandas as pd. modin.pandas is a 'drag and drop' replacement for pandas that enables the scale & performance benefits of the Snowpark Pandas APIs. 

There are some limitations however, when it comes to integrating with 3rd party libraries: (https://docs.snowflake.com/developer-guide/snowpark/python/pandas-on-snowflake#limitations)

---

Note that below, please avoid re-importing native pandas, after importing modin.pandas

# 3. Load data

In [ ]:
from snowflake.snowpark.functions import col
#CG note: commented out this line
#import pandas as pd
import snowflake.snowpark as snowpark

# Create a Snowpark session with a default connection.
from snowflake.snowpark.session import Session
session = Session.builder.configs(connection_parameters).create()

table_name = "NYC_YELLOW_TRIPS"

def load_data(session: snowpark.Session, database:str, schema:str, table_name:str): 
    '''loads data from a table exist in active session'''  
    
    # Step 1 :- Getting data from tables (Snowflake table)  
    source_table = f"{database}.{schema}.{table_name}" 
    

    # Step2: Create a snowpark DataFrame from source table.
    snowpark_df = session.table(source_table)
    
    # Step3: Create a Snowpark pandas DataFrame from existing Snowflake table
    #snowpark_pandas_df = pd.read_snowflake(source_table)

    # Step3: Convert snowpark Dataframe into your choice (pandas df, snowpark pandas, spark df)
    # Step3a: Converting the data to pandas dataframe
    pandas_df = snowpark_df.to_pandas()

    # converting a Pandas DataFrame into a Spark DataFrame.
    
    # return snowpark_df
    return pandas_df

## CG Demo of creating pandas dataframe

In [12]:
df = pd.DataFrame([[1, 'Big Bear', 8],[2, 'Big Bear', 10],[3, 'Big Bear', None],
                    [1, 'Tahoe', 3],[2, 'Tahoe', None],[3, 'Tahoe', 13],
                    [1, 'Whistler', None],['Friday', 'Whistler', 40],[3, 'Whistler', 25]],
                    columns=["DAY", "LOCATION", "SNOWFALL"])

In [13]:
df

,DAY,LOCATION,SNOWFALL
0,1,Big Bear,8.0
1,2,Big Bear,10.0
2,3,Big Bear,NaN
3,1,Tahoe,3.0
4,2,Tahoe,NaN
5,3,Tahoe,13.0
6,1,Whistler,NaN
7,Friday,Whistler,40.0
8,3,Whistler,25.0


Observe below the modin.pandas.dataframe type. This is a Snowpark pandas dataframe 

In [14]:
type(df)

modin.pandas.dataframe.DataFrame

In [22]:
df.columns

Index(['DAY', 'LOCATION', 'SNOWFALL'], dtype='object')

In [23]:
dfs = df.to_snowpark(index=False)

In [24]:
type(dfs)

snowflake.snowpark.dataframe.DataFrame

In [26]:
df_pandas = df.to_pandas()

Observe that .to_pandas() converts Snowpark pandas DF to native pandas df. Native pandas methods can then be used on this dataframe, directly via modin.pandas library

In [27]:
type(df_pandas)

pandas.core.frame.DataFrame

---
End (Colin Demo)
---

In [ ]:
# To print out the first 10 rows of snowpark df
import time
t0 = time.perf_counter()

snowpark_df= load_data(session,database,schema,table_name)
print(snowpark_df.show())

t1 = time.perf_counter()
print("snowpark_df printing 10 rows took:", t1-t0, "seconds")


In [ ]:

# To print out the first 10 rows of pandas df
import time
t0 = time.perf_counter()

pandas_df= load_data(session,database,schema,table_name)
print("Size of df:", pandas_df.shape)
print(pandas_df.head(5))

t1 = time.perf_counter()
print("pandas_df printing 10 rows took:", t1-t0, "seconds")

# 4. Transformations

- Features aggregated by location id and refreshed every 12 hours -> AVG_FARE_1H (float), AVG_FARE_10H (float)
- Features per trip refreshed every day - PASSENGER_COUNT, TRIP_DISTANCE, FARE_AMOUNT

## f_trip data

In [ ]:
import time
t0 = time.perf_counter()

f_trip_data = data_set[["TRIP_ID","PASSENGER_COUNT","TRIP_DISTANCE","FARE_AMOUNT"]]
print(f_trip_data.head())

t1 = time.perf_counter()
print("df.head printing took:", t1-t0, "seconds")

# and if you really want your answer in minutes:
#print(f"In minutes: {(t1-t0)/60}")



# Convert pandas df to snowpark df

In [ ]:
snowpark_df = session.create_dataframe(data=f_trip_data)
snowpark_df.head()

### f_location_data

In [ ]:
data_set.dtypes["TPEP_DROPOFF_DATETIME"]

In [ ]:
data_set["TPEP_DROPOFF_DATETIME"]

In [ ]:
data_set.dtypes["TPEP_DROPOFF_DATETIME"]

In [ ]:
data_set["reach_minutes"]= (data_set["TPEP_DROPOFF_DATETIME"]).dt.total_seconds()/60
data_set["reach_minutes"]= data_set["reach_minutes"].astype(int)
data_set.head()

In [ ]:
data_set.sort_values(by=["TPEP_DROPOFF_DATETIME"],ascending=False)[["TPEP_DROPOFF_DATETIME","DOLOCATIONID","trip_time_minutes","FARE_AMOUNT"]]


In [ ]:
AVG_FARE_1H= data_set.groupby(["DOLOCATIONID"]).apply(lambda x: "FARE_AMOUNT"]]
f_loc.head()

# Create multiple entities test

In [ ]:
import pandas as pd
df = pd.read_csv("entities_list.csv")
df

In [ ]:
import json
entities = json.load(open("entities.json", "r"))
entities

In [ ]:
for index,row in enumerate(df.values):
    print(index, row)

In [ ]:
for index,row in enumerate(df.values):
    for i in index:
        print(i)